In [1]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

In [2]:
pip show dlt

Name: dlt
Version: 1.12.3
Summary: dlt is an open-source python-first scalable data loading library that does not require any backend to run.
Home-page: https://github.com/dlt-hub
Author: 
Author-email: "dltHub Inc." <services@dlthub.com>
License-Expression: Apache-2.0
Location: /usr/local/python/3.12.1/lib/python3.12/site-packages
Requires: click, fsspec, gitpython, giturlparse, hexbytes, humanize, jsonpath-ng, orjson, packaging, pathvalidate, pendulum, pluggy, pytz, pyyaml, requests, requirements-parser, rich-argparse, semver, setuptools, simplejson, sqlglot, tenacity, tomlkit, typing-extensions, tzdata
Required-by: cognee
Note: you may need to restart the kernel to use updated packages.


In [10]:
import dlt
import requests

@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [14]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [15]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)

load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

2025-07-21 09:31:24,054|[WARNING]|26047|132885770990208|dlt|pipeline.py|_state_to_props:1680|The destination dlt.destinations.duckdb:None in state differs from destination dlt.destinations.qdrant:qdrant in pipeline and will be ignored
/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.34it/s]


Run started at 2025-07-21 09:31:24.065226+00:00 and COMPLETED in 11.92 seconds with 4 steps.
Step extract COMPLETED in 0.45 seconds.

Load package 1753090290.8106022 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.07 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1753090290.8106022 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 4.66 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.65 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-zoomcamp/dlt-workshop/db.qdrant location to store data
Load package 1753090290.8106022 is LOADED and contains no failed jobs

Step run COMPLETED in 11.92 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.65 seconds
1 load package(s) were loaded to des